In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import stats
from scipy import linalg, mat, dot
from collections import Counter
import string
import json
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import os
from ast import literal_eval

from file_operations import get_all_files, process_token

## Store Tokens

In [14]:
root = "../data/cnn_dailymail/data"
daily_path, cnn_path = "/dailymail/stories/", "/cnn/stories/"

In [16]:
def read_file(file_path):
    with open(root+daily_path+file_path, 'r') as file:
        
        content = file.read()
        content = content.split("@highlight")[0]
        
        # Tokenize the text
        result = process_token(content)

        return file_path, result

def token_all_files(path):
    file_paths = get_all_files(path)
    id_list=[]
    content_list=[]
    for path in file_paths:
        file_path,content=read_file(path)
        id_list.append(file_path)
        content_list.append(content)
        
    return pd.DataFrame({"id":id_list,'content': content_list})

In [4]:
result_df_cnn=token_all_files(root+cnn_path)

In [5]:
result_df_cnn

,id,content
0,638ba1352bdf405a8f5bd681d7fe5c928686afff.story,"[start, big, week, higgs, boson, soughtafter, ..."
1,f9f9601180ab3278165d936821e8f145659997f3.story,"[cnn, george, zimmerman, acquitted, florida, j..."
2,80ec0efb252ec4470aee44482d1e196111b5780b.story,"[cnn, zlatan, ibrahimovic, scored, third, goal..."
3,8435150be66ea9792999dfc233cc690f9c2fe2d0.story,"[cnn, nobel, laureate, norman, e, borlaug, agr..."
4,1444cf4d1832507a29a98529c2cd1a41f0154b52.story,"[cnn, louisiana, gov, bobby, jindal, monday, s..."
...,...,...
92573,d9699dc87c898b466b112fea8b48980e9df6e08d.story,"[weimar, germany, cnn, long, narrow, road, win..."
92574,9db26e940aa59bda5bf4244c32b30fdd7d07c357.story,"[james, theodorou, reflects, mistake, past, jo..."
92575,432986f7670e5fa46f4c2d46e696f8957d0fb8de.story,"[cnn, going, green, green, light, business, co..."
92576,4ecc129fd7231a641a2b545ecb5f667e94df6b66.story,"[prague, czech, republic, cnn, fireworks, rock..."


In [ ]:
result_df_dailymail=token_all_files(root+daily_path)
result_df_dailymail

In [7]:
result_df=pd.concat([result_df_cnn, result_df_dailymail], ignore_index=True)

In [11]:
result_df.iloc[:int(0.5*len(result_df)),:].to_csv('/Users/daiqiaochu/Desktop/ISE540/project/half_result1.csv', index=False)
result_df.iloc[int(0.5*len(result_df)):,:].to_csv('/Users/daiqiaochu/Desktop/ISE540/project/half_result2.csv', index=False)

#### entity

## Compute TFIDF

In [2]:
root = "../data/tf_idf/full_text/"
half_result=pd.read_csv(root+'half_result2.csv')
with open(root+'idf.json', 'r') as file:
    idf_file = json.load(file)
with open(root+'vocabulary.json', 'r') as file:
    vocabulary_file = json.load(file)

In [3]:
result_df=half_result

In [4]:
final_dic={}
for i in range(len(result_df)):
    final_dic[result_df.iloc[i,0]]=literal_eval(result_df.iloc[i,1])

In [5]:
import csv
import pandas as pd
from concurrent.futures import ProcessPoolExecutor
from collections import Counter
import concurrent.futures
import traceback

def calculate_TF(Seg_Docs, ID):
    if ID in Seg_Docs:
        term_counts=Counter(Seg_Docs[ID])
        TF_dic={term: count for term, count in term_counts.items()}
        return TF_dic
    else:
        print('Error: This key does not exist in the document')
        
def calculate_TFIDF(ID, Vocabulary, IDF_list):
    global final_dic
    Seg_Docs=final_dic
    non_zero_values = []
    non_zero_indices=[]
    TF_list = calculate_TF(Seg_Docs, ID)
    i=0
    for word in Vocabulary:
        if word in TF_list:
            non_zero_values.append(IDF_list[word] * TF_list[word])
            non_zero_indices.append(i)
            
        i+=1
    return non_zero_values,non_zero_indices

def process_and_save_row(ID):
    global vocabulary_file
    global idf_file
    Vocabulary=vocabulary_file
    nonzero_values, nonzero_positions= calculate_TFIDF(ID, Vocabulary, idf_file)
    row_values=[ID,nonzero_values,nonzero_positions]
    with open(root+'tfidf_result2.csv', 'a', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow([value for value in row_values])



In [6]:
keys = list(final_dic.keys())

with concurrent.futures.ThreadPoolExecutor() as executor:
    executor.map(process_and_save_row, keys)


In [ ]:
import numpy as np
from scipy.sparse import csr_matrix

# Create a two-column dataframe
df = pd.DataFrame({
    'values': [1, 2, 3, 1],
    'positions': [0, 1, 2, 8]
})

# Convert the dataframe back to a sparse matrix
original_matrix = csr_matrix((df['values'], df['positions'], [0, len(df)]), shape=(1, 11))

print("Original Matrix:")
print(original_matrix.toarray())